In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from training.dataset import TrainDataset, TestDataset, img_transform

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
models = {'vgg11_v1' : '/opt/ml/code/trained_models/vgg11_batchsize128_lr001_epoch30.pt',
          'vgg11_v2' : '/opt/ml/code/trained_models/vgg11_batchsize128_lr001_epoch7.pt',
          'vgg11_v3' : '/opt/ml/code/trained_models/vgg11_batchsize128_lr001_epoch30CenterCrop.pt',
          'vgg11_v4' : '/opt/ml/code/trained_models/vgg11_batchsize128_lr001_epoch30_CenterCropageAug.pt',
          'vgg11_v5' : '/opt/ml/code/trained_models/vgg11_batchsize128_lr001_epoch60_CenterCrop_ageAug.pt',
          'vgg11_v6' : '/opt/ml/code/trained_models/vgg11_batchsize128_lr001_epoch15_CenterCrop.pt',
          'vgg11_v7' : '/opt/ml/code/trained_models/vgg11_batchsize128_lr001_epoch15_resize_CenterCrop.pt',
          'vgg11_v8' : '/opt/ml/code/trained_models/vgg11_batchsize128_lr001_epoch45_resize_CenterCrop_scheduler.pt',
          'effnetb0_v1' : '/opt/ml/code/trained_models/effnetb0_batchsize128_lr001_epoch5_resize_CenterCrop.pt',
          'effnetb4_v1' : '/opt/ml/code/trained_models/effnetb4_batchsize128_lr001_epoch5_resize_CenterCrop.pt',
          'effnetb4_v2' : '/opt/ml/code/trained_models/effnetb4_batchsize128_lr001_epoch40_resize_CenterCrop.pt',
          'effnetb4_v3' : '/opt/ml/code/trained_models/effnetb4_batchsize128_lr0001_epoch40_CenterCrop_scheduler_nofreeze.pt',
          'effnetb4_v4' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_40-CenterCrop-scheduler-nofreeze-fix_data.pt',
          'effnetb4_v5' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_120-CenterCrop-scheduler-nofreeze-fix_data.pt',
          'effnetb4_v6' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_120-CenterCrop-scheduler-nofreeze-fix_data-drop_mask_sample0.6.pt',
          'effnetb4_v7' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_60-CenterCrop-scheduler-nofreeze-fix_data.pt',
          'effnetb4_v8' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_40-CenterCrop-scheduler-nofreeze-fix_data-novalid.pt',
          'effnetb4_v9' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_60-CenterCrop-scheduler-nofreeze-fix_data-focal_loss.pt',
          'effnetb4_v10' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_10-CenterCrop-scheduler-nofreeze-fix_data-focal_loss.pt',
          'effnetb4_v11' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_10-CenterCrop-scheduler-nofreeze-fix_data-focal_loss.pt',
          'effnetb4_v12' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_60-CenterCrop-scheduler-nofreeze-fix_data-focal_loss_gamma4.pt',
          'effnetb4_v13' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_60-CenterCrop-scheduler-nofreeze-fix_data-no_overlap.pt',
          'effnetb4_v14' : '/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_60-CenterCrop-scheduler-nofreeze-fix_data-no_overlap-focal_loss.pt',
          'effnetb0_v2' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_001-epoch_20-CenterCrop-cosin_warm_restart-nofreeze-fix_data-no_overlap.pt',
          'effnetb0_v3' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-CenterCrop-cosin_warm_restart-nofreeze-fix_data-no_overlap-focal_loss_gamma5.pt',
          'effnetb0_v4' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-CenterCrop-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58.pt',
          'effnetb0_v5' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-resize-CenterCrop-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58.pt',
          'effnetb0_v6' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-CenterCrop-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05.pt',
          'effnetb0_v7' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-CenterCrop-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05-rotate10.pt',
          'effnetb0_v8' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-CenterCrop-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05resize.pt',
          'effnetb0_v9' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-CenterCrop400_250-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05-resize224.pt',
          'effnetb0_v10' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-CenterCrop400_250-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05.pt',
          'effnetb0_v11' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-CenterCrop400_250-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05-weighted_sampler.pt',
          'effnetb0_v12' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05.pt',
          'effnetb0_v13' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_40-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05-center_crop500250-LS.pt',
          'effnetb0_v14' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_40-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05-center_crop500250-LS_alpha02.pt',
          'effnetb0_v15' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_20-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05-center_crop500250-LS_alpha04.pt',
          'effnetb0_v16' : '/opt/ml/code/trained_models/effnetb0-batchsize_128-lr_0001-epoch_15-cosin_warm_restart-nofreeze-fix_data-no_overlap-age_aug58-Hflip05-center_crop500250-LS_alpha06.pt'}
          
model_to_use = 'effnetb0_v16'

In [3]:
test_dir = '/opt/ml/input/data/eval'
sub_dir = '/opt/ml/code/submission/'
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')







In [4]:
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = img_transform(train=False)

In [5]:
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False,
    num_workers = 4
)

In [6]:
# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)



device = torch.device('cuda')
model = torch.load(models[model_to_use]).to(device)

model.eval()
print('haha')

haha


In [7]:
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
sub_name = os.path.basename(models[model_to_use]).rstrip('.pt')
submission.to_csv(os.path.join(sub_dir, sub_name + '.csv'), index=False)
print('test inference is done!')

test inference is done!
